In [195]:
import sys
sys.path.append("../")
sys.path

['d:\\projects\\moshack\\notebooks',
 'c:\\Users\\diogenbip\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip',
 'c:\\Users\\diogenbip\\AppData\\Local\\Programs\\Python\\Python310\\DLLs',
 'c:\\Users\\diogenbip\\AppData\\Local\\Programs\\Python\\Python310\\lib',
 'c:\\Users\\diogenbip\\AppData\\Local\\Programs\\Python\\Python310',
 '',
 'c:\\Users\\diogenbip\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages',
 'c:\\Users\\diogenbip\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32',
 'c:\\Users\\diogenbip\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\diogenbip\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\Pythonwin',
 '../',
 '../',
 '../',
 '../',
 '../']

In [196]:
import random 
import librosa
import numpy as np
import matplotlib.pyplot as plt
import libs.AudioDataSet

In [197]:
def jaccard_similarity(signature1, signature2):
    """
    Compute the Jaccard similarity between two MinHash signatures.

    Parameters:
    signature1 (numpy array): MinHash signature of shape (num_hashes,)
    signature2 (numpy array): MinHash signature of shape (num_hashes,)

    Returns:
    similarity (float): Jaccard similarity between the two signatures
    """
    intersection = np.sum(np.bitwise_and(signature1, signature2))
    union = np.sum(np.bitwise_or(signature1, signature2))
    return intersection / union

def minHASH(arrays, num_perm):
  permutations = []
  count = 0
  while count < num_perm:
    perm = random.sample(range(0, len(arrays[0])), len(arrays[0]))
      # print(skm.mutual_info_score(arrays[0], perm))
    permutations.append(perm)
    count += 1
    
  signatures = []
  for i in range(len(arrays)):
    signature = []
    for permutation in permutations:
      for j in permutation:
        if(arrays[i][j] == 1):
          signature.append(j)
          break
    signatures.append(signature)
  return signatures
      

In [198]:
class MinHash:
  def __init__(self, num_perm):
    self.permutations = self._permutations_generate(num_perm)
  
  def _permutations_generate(self, num_perm)->list:
    permutations = []
    count = 0
    while count < num_perm:
      perm = random.sample(range(0, 255), 100)
      permutations.append(perm)
      count += 1
    return permutations
      
  def hash(self,array)->list:
    signature = []
    for permutation in self.permutations:
      for j in permutation:
        if(array[j] == 1):
          signature.append(j)
          break
    return signature
  
  def jaccard_similarity(self,signature1, signature2):
    intersection = np.sum(np.bitwise_and(signature1, signature2))
    union = np.sum(np.bitwise_or(signature1, signature2))
    return intersection / union 

In [199]:
class LSH:
    
    def __init__(self, threshold:float,num_hash_functions:int = 100,bands:int = 25, rows:int = 4):
      assert threshold > 0 and threshold < 1
      assert bands*rows == num_hash_functions
      self._threshold = threshold
      self._num_hash_functions = num_hash_functions
      self._bands = bands
      self._rows = rows
      self._min_hash = MinHash(num_hash_functions)
      self._buckets = {}
    
    def clear(self):
      self._buckets.clear()

    def search(self, fingerprints,name,time, add_to_bucket = True)->tuple :
        min_hashes = self._min_hash.hash(fingerprints)
        band_hashes = []
        compared_sketches = set()
        
        if(len(min_hashes) != self._num_hash_functions):
          return None
        
        for i in range(self._bands):
          band_hashes.append(self._compute_band_hash(min_hashes, i))
          if band_hashes[i] in self._buckets:
            for sketch_to_check in self._buckets[band_hashes[i]]:
              check_key = ''.join(str(x) for x in sketch_to_check)
              if check_key not in compared_sketches:
                jac = self._min_hash.jaccard_similarity(sketch_to_check[0], min_hashes)
                if jac >= self._threshold:
                  return (sketch_to_check[0],sketch_to_check[1],sketch_to_check[2], jac)
                compared_sketches.add(check_key)
        if add_to_bucket:
          for i in range(self._bands):
            if band_hashes[i] not in self._buckets:
              self._buckets[band_hashes[i]] = []
            self._buckets[band_hashes[i]].append((min_hashes, name, time))
          
    def _compute_band_hash(self, min_hashes: list, i: int) -> str:
        """Compute a hash for quick bucket match search."""
        band_hash_list = []
        for j in range(self._rows):
            # Adding the rows corresponding to ith band
            band_hash_list.append('%02d' % min_hashes[i * self._rows + j])

        # Adding the number i to distinguish between bands
        band_hash_list.append('%02d' % i)
        return ''.join(band_hash_list)
      
    def get_min_hash(self):
      return self._min_hash
      

In [200]:
def get_32bin_mel(samples):
  return librosa.feature.melspectrogram(y=samples, sr=5512,n_fft=2048, hop_length=64, n_mels=32)

In [201]:
def get_energy_spec(spec):
  # Transpose 'spec' upfront and get the shifted versions for calculations
  specT = spec.T
  specT_shifted_vertically = np.roll(specT, shift=-1, axis=0)
  specT_shifted_horizontally = np.roll(specT, shift=-1, axis=1)

  # Calculate the differences for current and previous rows
  diff_current = specT - specT_shifted_horizontally
  diff_previous = specT_shifted_vertically - np.roll(specT_shifted_horizontally, shift=-1, axis=0)

  # Compute the energy_spec matrix using NumPy's greater function for element-wise comparison
  energy_spec = np.greater(diff_current - diff_previous, 0).astype(int)
  return energy_spec.T

In [202]:
dataSet = libs.AudioDataSet.AudioDataset("../data/compressed_index/")




In [203]:
audio_1 = dataSet[0]


In [204]:
audio_2 = dataSet[198]

In [205]:
audio_3 = dataSet[1]

In [206]:
from moviepy.audio.AudioClip import AudioArrayClip
audio2 = AudioArrayClip(audio_2.T, fps=5512*2)
audio2.write_audiofile('test_2.wav')

MoviePy - Writing audio in test_2.wav


MoviePy - Done.


In [207]:
energy_spec1 = get_energy_spec(get_32bin_mel(audio_1[0]))
energy_spec2 = get_energy_spec(get_32bin_mel(audio_2[0]))

In [208]:
fingerprint1 = []
for i in range(0,energy_spec1.shape[1],128):
  fingerprint1.append(energy_spec1[:,i:i+128].flatten())
fingerprint2 = []
for i in range(0,energy_spec2.shape[1],128):
  fingerprint2.append(energy_spec2[:,i:i+128].flatten())


In [209]:
lsh = LSH(0.81)

In [210]:
for i,f in enumerate(fingerprint1):
  lsh.search(f,"First video",i)

In [211]:
sims = {}
for i,f in enumerate(fingerprint2):
  s = lsh.search(f,"Second video",i,add_to_bucket=False)
  if(s is not None):
    # print("This time: ", i*1.48)
    # print(s[1:])
    if i in sims:
      sims[i].append(s)
    else:
      sims[i] = [s]

In [212]:
sotred = dict(sorted(sims.items(), key=lambda x: x[1][0][3]))

In [213]:
def get_sync_frame(key, minhash):
  sec_30 = 30 * 5512 // 64
  check_frame = key*128
  minHash = lsh.get_min_hash()
  for i in range(check_frame - sec_30, check_frame + sec_30):
    fingerprint = energy_spec2[:,i:i+128].flatten()
    if len(fingerprint) != 128:
      continue
    fingerprint = minHash.hash(fingerprint)
    jac = minHash.jaccard_similarity(fingerprint, minhash)
    if(jac > 0.98):
      return i
  return -1





In [214]:
sync_frames = []
for key,value in sotred.items():
  frame = get_sync_frame(key, value[0][0])
  if(frame != -1):
    sync_frames.append(frame)

IndexError: index 156 is out of bounds for axis 0 with size 128

In [ ]:
sync_frames

[]

In [ ]:


sync_frame = get_sync_frame(sotred)
sync_fingerprint = []
if(sync_frame != -1):
  for i in range(sync_frame,energy_spec2.shape[1],128):
    sync_fingerprint.append(energy_spec2[:,i:i+128].flatten())

In [ ]:
ranges = []
range_i = []
for i,f in enumerate(sync_fingerprint):
  s = lsh.search(f,"Second video",i,add_to_bucket=False)
  if(s is not None):
    # this_min, this_sec = divmod((i+sync_frame//128)*1.48, 60)
    # this_time_str = '{:02d}:{:02d}'.format(int(this_min), int(this_sec)) 
    # target_min, target_sec = divmod(s[2]*1.48, 60)
    # target_time_str = '{:02d}:{:02d}'.format(int(target_min), int(target_sec))
    if(len(range_i) == 0):
      range_i = [i+sync_frame//128, s[2]]
    else:
      #430 - 5 sec
      if(s[2] - range_i[1]< 430):
        range_i[1] = s[2]
      else:
        ranges.append(range_i)
        range_i = [i+sync_frame/128, s[2]]

if(len(range_i) > 0):
  ranges.append(range_i)
    
    # print(s[1:])
print(ranges)


[[1022, 1086]]
